## Option pricing

Text.

---

---

### Black-Scholes model

The Black-Scholes model allows to compute the theoretical value of an option contract for European options, given a stock $S(t) is modelled using a Brownian SDE, based on five input variables:

![](./img/bs-model-overview.png)

for which the dynamics are given by

$$
	dS(t) = ...
	\\
	dV(t) = ...
$$

and an analytical solution can be obtained using the Feynman-Kac theorem, as

$$
	V(t) = ...
$$